In [1]:
import os
import tensorflow as tf 
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from shutil import copyfile
import matplotlib.pyplot as plt
import splitfolders

In [2]:
# base_dir = '\capstone\Dataset'
base_dir = '/capstone/Dataset'

In [3]:
splitfolders.ratio(
    base_dir,
    output='fixDataset',
    ratio=(.7,0.1, 0.2)
)

Copying files: 1500 files [00:07, 211.82 files/s]


In [4]:
train_dir = 'fixDataset/train/'
val_dir = 'fixDataset/val/'
val_dir = 'fixDataset/test/'
os.listdir(train_dir)

['Angklung', 'Arumba', 'Calung', 'Jengglong', 'Kendang']

In [5]:
# augmentasi gambar
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=0.45, # memutar gambar
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    brightness_range=(0.5,1.0), # value < 1 lebih gelap
                    shear_range=0.2, # untuk mengatur skala image
                    zoom_range=(0.8,1.0), # value < 1 akan zoom in
                    fill_mode='nearest', # untuk mengisi gambar atau wadah yang tidak memiliki nilai
                    horizontal_flip=True
                    )
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_dir, # direktori data train
    target_size=(150,150), # mengubah resolusi seluruh gambar jadi 150*150
    batch_size=32,# untuk menentukan jumlah image yang akan dimasukkan ke dalam steps training
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

Found 1198 images belonging to 5 classes.
Found 304 images belonging to 5 classes.
Found 304 images belonging to 5 classes.


In [7]:
image, label = train_generator.next()

In [8]:
class_name = os.listdir(train_dir)

In [9]:
class_name

['Angklung', 'Arumba', 'Calung', 'Jengglong', 'Kendang']

MODEL

In [27]:
from keras.callbacks import EarlyStopping

monitor_val_acc = EarlyStopping(monitor='val_accuracy', patience=3)

In [28]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)), #input layer
    # Hidden Layer
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(), #menyatukan inputan yang memiliki dimensi
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    # Hidden Layer
    tf.keras.layers.Dense(5, activation='softmax') # output Layer
])

In [29]:
model.compile(
    tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [30]:
history = model.fit(
    train_generator,
    shuffle=True,
    verbose=True,
    epochs=30,
    validation_data=val_generator,
    callbacks=[monitor_val_acc]
)

Epoch 1/30
38/38 [==============================] - 81s 2s/step - loss: 1.5551 - accuracy: 0.2538 - val_loss: 1.5413 - val_accuracy: 0.3289
Epoch 2/30
38/38 [==============================] - 78s 2s/step - loss: 1.2845 - accuracy: 0.4424 - val_loss: 2.1619 - val_accuracy: 0.2467
Epoch 3/30
38/38 [==============================] - 76s 2s/step - loss: 1.0898 - accuracy: 0.5584 - val_loss: 1.3373 - val_accuracy: 0.5132
Epoch 4/30
38/38 [==============================] - 98s 3s/step - loss: 0.9670 - accuracy: 0.6152 - val_loss: 1.5578 - val_accuracy: 0.4704
Epoch 5/30
38/38 [==============================] - 82s 2s/step - loss: 0.9151 - accuracy: 0.6469 - val_loss: 1.0248 - val_accuracy: 0.6118
Epoch 6/30
38/38 [==============================] - 70s 2s/step - loss: 0.7105 - accuracy: 0.7412 - val_loss: 1.1673 - val_accuracy: 0.6316
Epoch 7/30
38/38 [==============================] - 71s 2s/step - loss: 0.6457 - accuracy: 0.7504 - val_loss: 0.7078 - val_accuracy: 0.7138
Epoch 8/30
38/38 [==

In [31]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    patience= 2,
    verbose= 1,
    factor=0.3,
    min_lr=0.000001
)

In [32]:
from tensorflow.keras.applications import InceptionV3
model = InceptionV3(weights = 'imagenet',
                    include_top=False,
                    input_shape=(150,150,3))
model.trainable = False

In [33]:
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications

In [34]:
add_model = Sequential()
add_model.add(model)
add_model.add(Flatten())
add_model.add(tf.keras.layers.Dense(1024, activation='relu'))
add_model.add(tf.keras.layers.Dense(512, activation='relu'))
add_model.add(tf.keras.layers.Dense(256, activation='relu'))
add_model.add(Dropout(0.2))
add_model.add(Dense(5, activation='softmax'))

model = add_model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 flatten_5 (Flatten)         (None, 18432)             0         
                                                                 
 dense_17 (Dense)            (None, 1024)              18875392  
                                                                 
 dense_18 (Dense)            (None, 512)               524800    
                                                                 
 dense_19 (Dense)            (None, 256)               131328    
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_20 (Dense)            (None, 5)                

c:\Users\ahmad\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [35]:
history = model.fit(train_generator,
                    epochs=15,
                    shuffle=True,
                    verbose=True,
                    validation_data=val_generator,
                    callbacks=[reduce_lr,monitor_val_acc])

Epoch 1/15
38/38 [==============================] - 87s 2s/step - loss: 0.6811 - accuracy: 0.7880 - val_loss: 0.1115 - val_accuracy: 0.9474 - lr: 1.0000e-04
Epoch 2/15
38/38 [==============================] - 72s 2s/step - loss: 0.1385 - accuracy: 0.9524 - val_loss: 0.0309 - val_accuracy: 0.9868 - lr: 1.0000e-04
Epoch 3/15
38/38 [==============================] - 70s 2s/step - loss: 0.0615 - accuracy: 0.9741 - val_loss: 0.0241 - val_accuracy: 0.9934 - lr: 1.0000e-04
Epoch 4/15
38/38 [==============================] - 70s 2s/step - loss: 0.0809 - accuracy: 0.9783 - val_loss: 0.0211 - val_accuracy: 0.9868 - lr: 1.0000e-04
Epoch 5/15
38/38 [==============================] - 70s 2s/step - loss: 0.0603 - accuracy: 0.9800 - val_loss: 0.0028 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 6/15
38/38 [==============================] - 70s 2s/step - loss: 0.0505 - accuracy: 0.9816 - val_loss: 0.0054 - val_accuracy: 0.9967 - lr: 1.0000e-04
Epoch 7/15
38/38 [==============================] - ETA: 0

In [36]:
add_model.save('Model.h5')